In [1]:
import json
import os
from ensure import ensure_annotations
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth',None)
import matplotlib.pyplot as plt

In [2]:
print((os.getcwd()))
os.chdir('d:\\vscode_machineLearning\\BEST_PROJECTS\\Question-Answering-system-LLMs')
print((os.getcwd()))

d:\vscode_machineLearning\BEST_PROJECTS\Question-Answering-system-LLMs\notebook
d:\vscode_machineLearning\BEST_PROJECTS\Question-Answering-system-LLMs


In [3]:
@ensure_annotations
def random_values(feature_name:str,total=6):
    for i in range(0,total):
        index = np.random.randint(df.shape[0])
        print(index,':',df[feature_name].iloc[index])
            

In [4]:
@ensure_annotations
def random_values_df(col_name:str,df:pd.DataFrame,total=6):
    for i in range(0,total):
        index = np.random.randint(df.shape[0])
        print(index,':',df[col_name].iloc[index])        

In [5]:
df = pd.read_csv(filepath_or_buffer=r'data/S08_question_answer_pairs.txt',sep='\t')

In [6]:
df = df[['Question','Answer']]

In [7]:
df.isnull().sum()

Question     19
Answer      242
dtype: int64

In [8]:
df.dropna(inplace=True)

In [9]:
df.sample(5)

,Question,Answer
520,"According to Reader's Digest, is Finland best for living?",Yes
174,Are certain species of beetles considered pests?,Yes
320,What is the largest country in the world?,Russia.
1675,Where was the League of Nations created?,Paris
9,When did Lincoln begin his political career?,1832.


In [14]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


# nltk.download("punkt")


In [11]:
df = Dataset.from_pandas(df)

In [12]:
train_test_df = df.train_test_split()

In [ ]:
train_test_df

DatasetDict({
    train: Dataset({
        features: ['Question', 'Answer', '__index_level_0__'],
        num_rows: 1104
    })
    test: Dataset({
        features: ['Question', 'Answer', '__index_level_0__'],
        num_rows: 369
    })
})

In [ ]:
train_test_df=train_test_df.remove_columns(column_names='__index_level_0__')

In [ ]:
train_test_df

DatasetDict({
    train: Dataset({
        features: ['Question', 'Answer'],
        num_rows: 1104
    })
    test: Dataset({
        features: ['Question', 'Answer'],
        num_rows: 369
    })
})

## Tokenization

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering

In [ ]:
checkpoint = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(checkpoint)
model = DistilBertForQuestionAnswering.from_pretrained(checkpoint)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to

In [ ]:
print(f"Vocab size is : {tokenizer.vocab_size}")

print(f"Model max length is : {tokenizer.model_max_length}")

print(f"Model input names are: {tokenizer.model_input_names}")

Vocab size is : 30522
Model max length is : 512
Model input names are: ['input_ids', 'attention_mask']


In [ ]:
train_test_df['train']['Question']

["Is a beetle's general anatomy uniform?",
 'What established a trading post on the island in 1819?',
 'Is Canada bilingual?',
 'Was Woodrow Wilson the thirtieth President of the United States?',
 'What happened in recent years?',
 'Did Uruguay host the first ever World Cup?',
 "Is it the world 's fourth most populous country and the most populous muslim-majority nation?",
 'How much of the population is of white European descent?',
 'Was Ford a member of the House of Representatives?',
 'What is the official language of Liechtenstein?',
 'The Historical Society of the Principality of Liechtenstein plays what?',
 'Is Ford one of only four former Presidents to live to 90 or more years of age ?',
 'Are turtles a part of the Flying Spaghetti Monster?',
 'Was Lincoln chosen as a presidential candidate in 1860?',
 'What unrelated water birds are ducks sometimes confused with?',
 "Are Canada 's official national sports ice hockey ( winter ) and lacrosse ( summer ) ?",
 'Have thumbpolar bears

In [ ]:
def tokenize_dataset(examples):
    tokenized_data = tokenizer(examples['Question'],examples['Answer'], padding='longest', truncation=True)
    return {'input_ids': tokenized_data['input_ids'], 'attention_mask': tokenized_data['attention_mask']}

In [ ]:
train_tokenized = train_test_df['train'].map(tokenize_dataset)
test_tokenized = train_test_df['test'].map(tokenize_dataset)

Map:   0%|          | 0/1104 [00:00<?, ? examples/s]

Map:   0%|          | 0/369 [00:00<?, ? examples/s]

In [ ]:
train_tokenized['input_ids'][0]

[101, 2003, 1037, 7813, 1005, 1055, 2236, 13336, 6375, 1029, 102, 2748, 102]

In [ ]:
test_tokenized

Dataset({
    features: ['Question', 'Answer', 'input_ids', 'attention_mask'],
    num_rows: 369
})

## Training

In [ ]:
num_epochs = 3
batch_size = 32





In [ ]:
train_test_df['train']['Question']

['Who did James Monroe live with in New York City?',
 'Is a polar bear white in color?',
 'Who did Cleveland run against in 1884?',
 'Is an Asian elephant smaller than an African elephant?',
 'When did Canada have one of the largest armed forces in the world?',
 "Where is Qatar's telecommunication system centered?",
 'How many territories joined to form Romania?',
 'What do turles use to breathe in the water?',
 "What are the elephant's ears important for?",
 "Is Uruguay 's oldest church in San Carlos , Maldonado ?",
 'How many regions is Ghana divided into?',
 'What method of locomotion do Kangaroos Use?',
 'Is it not even known whether the gigantic palaeeudyptines constitute a monophyletic lineage , or whether gigantism was evolved independently in a much restricted Palaeeudyptinae and the Anthropornithinae ?',
 'Who did Millard Fillmore fall in love with?',
 "What was the nickname of Theodore Roosevelt's sister Anna?",
 'What did Jefferson call John Adams?',
 'Does a male leopard we